In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a=1
fluxes = []

for s,path_direction in enumerate([
                        r"H:\phd stuff\tidy3d\data\08_22_2024 chi 0.2 N484 T(t)\z_incidence\0.28-0.31",
                        r"H:\phd stuff\tidy3d\data\08_22_2024 chi 0.2 N484 T(t)\z_incidence\0.12-0.15",
                        # r"H:\phd stuff\tidy3d\data\08_22_2024 chi 0.2 N484 T(t)\z_incidence\0.144-0.18",
                        # r"H:\phd stuff\tidy3d\data\08_22_2024 chi 0.2 N484 T(t)\z_incidence\0.1-0.12",
                        # r"H:\phd stuff\tidy3d\data\08_23_2024 chi 0.45 N484 T(t)\z_incidence\0.28-0.31",
                        # r"H:\phd stuff\tidy3d\data\08_23_2024 chi 0.45 N484 T(t)\z_incidence\0.12-0.15",
                        # r"H:\phd stuff\tidy3d\data\08_23_2024 chi 0.45 N484 T(t)\z_incidence\0.1 - 0.12"
                       ]):

      folder_path = f"{path_direction}"
      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
           
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                #print(structure_1.__str__())
                sim_data = structure_1.sim_data
                flux_t = sim_data["time_monitorT"].flux
                times = flux_t['t']
                max_index = np.argmax(flux_t.values)
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[2].freqs)
                flux_item = {
                      "flux_t":flux_t,
                      "times":times,
                      "monitor_lambdas":monitor_lambdas,
                      "max_index":max_index,
                      "type":"Localization" if s == 0 else "Diffusive"
                }

                fluxes += [flux_item]
                
                plt.plot(times[:]*1e12,(flux_t[:]/np.max(flux_t)),"o",label=f"$\\nu$: {1/monitor_lambdas[0]:.3g} - {1/monitor_lambdas[-1]:.3g}")
      

                

      plt.ylabel("Normalized Transmitted Flux T(t)")
      plt.xlabel("Time[ps]")
      plt.title("Average transmission T(t)")
      plt.legend()
      plt.yscale("log")
      plt.grid()
      plt.show()


In [ ]:
flux_clean = []
add = True
for item in fluxes:
    if item["type"]=="Localization":
        # print(np.shape(item["times"]))
        if np.shape(item["times"])[0] == 800:
            flux_clean += [item]
    if item["type"]=="Diffusive":
        # print(np.shape(item["times"]))
        if np.shape(item["times"])[0] == 393:
            flux_clean += [item]


# flux_clean=np.copy(fluxes)
        

In [ ]:
fluxes_loc =([np.array(item["flux_t"]) for item in flux_clean if item["type"]=="Localization"])
fluxes_diff = ([np.array(item["flux_t"]) for item in flux_clean if item["type"]=="Diffusive"])
times_diff = flux_clean[-1]["times"]
times_loc = flux_clean[0]["times"]
monitor_lambdas_diff=flux_clean[-1]["monitor_lambdas"]
monitor_lambdas_loc=flux_clean[0]["monitor_lambdas"]

In [ ]:
loc = np.abs((np.mean(fluxes_loc,axis=0)))
max_index = np.argmax(loc)
diff = np.abs(np.mean(fluxes_diff,axis=0))
max_index_diff = np.argmax(diff)

In [ ]:
def moving_average(x, w=3):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
bad_loc_data_log = np.where(np.isnan(np.log(loc)))
cleaned_loc_data_log = np.delete(loc,bad_loc_data_log)
cleaned_loc_data_time = np.delete(times_loc,bad_loc_data_log)

In [ ]:
bad_diff_data_log = np.where(np.isnan(np.log(diff)))
cleaned_diff_data_log = np.delete(diff,bad_diff_data_log)
cleaned_diff_data_time = np.delete(times_diff,bad_diff_data_log)

In [ ]:
loc_moving_avg = moving_average((cleaned_loc_data_log),50)
diff_moving_avg = moving_average((cleaned_diff_data_log),3)

In [ ]:
plt.figure(figsize=(15, 10))
times_avg=np.array(cleaned_loc_data_time[max_index+49:]*1e12)
loc_avg=(loc_moving_avg[max_index:]/np.max(loc_moving_avg[max_index:]))
plt.plot(times_loc[max_index:]*1e12,((loc[max_index:])/np.max((loc[max_index:]))),"o",label=f"$\\nu$: {1/monitor_lambdas_loc[0]:.3g} - {1/monitor_lambdas_loc[-1]:.3g}")
plt.plot(times_avg,loc_avg,label=f"Moving Average")
plt.ylabel("T(t)")
plt.xlabel("Time[ps]")
plt.yscale("log")
plt.title("Average transmission T(t)")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
times_diff_avg=np.array(times_diff[max_index_diff+2:]*1e12)
diff_avg=(diff_moving_avg[max_index_diff:]/np.max(diff_moving_avg[max_index_diff:]))
plt.plot(times_diff[max_index_diff:]*1e12,(diff[max_index_diff:]/np.max(diff[max_index_diff:])),"o",label=f"$\\nu$: {1/monitor_lambdas_diff[0]:.3g} - {1/monitor_lambdas_diff[-1]:.3g}")
plt.plot(times_diff_avg,diff_avg,label=f"Moving Average")
plt.ylabel("T(t)")
plt.xlabel("Time[ps]")
plt.yscale("log")
plt.title("Average transmission T(t)")
plt.legend()
plt.show()

In [ ]:
import jax
import jax.numpy as jnp
from jaxopt import ScipyMinimize
import jaxopt
import matplotlib.pyplot as plt
jax.config.update("jax_enable_x64", True)

In [ ]:
def diffussive(t,A,C):
    B = jnp.exp(-A*t)
    return jnp.log(B)+C

In [ ]:
def trans_t_1(t,alpha,p,C): 
    
    B =( -p*jnp.log(t)-alpha*t)+C

    return (B)

In [ ]:
plt.figure(figsize=(15, 10))
model = lambda params,t: trans_t_1(t,params[0],params[1],params[2])
def loss_fn(params):
 y_pred = model(params, times_avg)
 return jnp.mean((y_pred - jnp.log(loc_avg))**2)

initial_params = jnp.array([0.5,0.85,3.5], dtype=jnp.float32)
optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=20000)
# Optimize the parameters
result = optimizer.run(initial_params)
optimal_params = result.params

print("Anderson: ",optimal_params)

transmission_anderson = trans_t_1(times_avg,optimal_params[0],optimal_params[1], optimal_params[2])
plt.plot(times_avg,np.exp(transmission_anderson),label="Anderson Fitted")


#####################Diff#########################
model = lambda params,t: diffussive(t,params[0],params[1])
def loss_fn(params):
 y_pred = model(params, times_avg[:])
 return jnp.mean((y_pred - jnp.log(loc_avg[:]))**2)

initial_params = jnp.array([0.5,0], dtype=jnp.float32)
optimizer = jaxopt.ScipyMinimize(fun=loss_fn,maxiter=15000)

# Optimize the parameters
result = optimizer.run(initial_params)
optimal_params = result.params

print("Diff: ",optimal_params)

transmission_diff = diffussive(times_avg,0.05,0.32)
plt.plot(times_avg,np.exp(transmission_diff),label="Exponential Fit")

#################################################





plt.plot(times_avg,(loc_avg),label=f"$\\nu$: {1/monitor_lambdas_loc[0]:.3g} - {1/monitor_lambdas_loc[-1]:.3g}")
plt.ylabel("T(t)")
plt.xlabel("Time[ps]")
plt.legend()
plt.yscale("log")
plt.title("Average transmission T(t)")
plt.grid()
plt.show()

In [ ]:
def diffussive(t,A,C):
    B = jnp.exp(-A*t)
    return jnp.log(B)+C


plt.figure(figsize=(15, 10))
indices = np.where(times_diff_avg>=20)
model = lambda params,t: diffussive(t,params[0], params[1])
def loss_fn(params):
 y_pred = model(params, times_diff_avg[indices])
 return jnp.mean((y_pred - jnp.log(diff_avg[indices]))**2)

initial_params = jnp.array([0.1,3.5], dtype=jnp.float32)
optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=20000)

# Optimize the parameters
result = optimizer.run(initial_params)
optimal_params = result.params

print("Diff: ",optimal_params)

transmission_diff = diffussive(times_diff_avg[:],optimal_params[0],optimal_params[1])
plt.plot(times_diff_avg[:],np.exp(transmission_diff),label="Diffusive Fitted")
plt.plot(times_diff_avg[:],(diff_avg[:]),label=f"$\\nu$: {1/monitor_lambdas_diff[0]:.3g} - {1/monitor_lambdas_diff[-1]:.3g}")
plt.ylabel("T(t)")
plt.xlabel("Time[ps]")
plt.title("Average transmission T(t)")
plt.yscale("log")
plt.legend()
plt.grid()
plt.show()